In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import numpy as np
import os.path as osp
import matplotlib.pylab as plt
from sklearn.manifold import TSNE

from general_tools.in_out.basics import unpickle_data
from general_tools.in_out import create_dir
from general_tools.notebook.tf import reset_tf_graph

from general_tools.clustering.plt import plot_2d_embedding_in_grid_greedy_way, \
                                         plot_2d_embedding_in_grid_forceful


    
from geo_tool import Point_Cloud

import tf_lab.point_clouds.in_out as pio

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import PointCloudDataSet

from tf_lab.autopredictors.scripts.virt_scan_data import plotting_default_params, pc_sampler, pc_loader
from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
top_data_dir = top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'

In [5]:
# load AE model
class_name = 'chair'
syn_id = shape_net_category_to_synth_id()[class_name]
ae_loss = 'emd'
ae_id = '12' # 128 bottlεneck

ae_net_name = 'ae_farm_' + class_name + '_conv_arch_' + ae_id + '_2048pts_' + ae_loss
ae_net_dir = osp.join(top_data_dir, 'OUT/icml/nn_models/ae_farming', ae_net_name)
ae_conf = Conf.load(osp.join(ae_net_dir, 'configuration'))
reset_tf_graph()
ae = PointNetAutoEncoder(ae_net_name, ae_conf)

saved_epochs = read_saved_epochs(ae_conf.train_dir)
load_epoch = saved_epochs[-1]
ae.restore_model(ae_conf.train_dir, load_epoch, verbose=True)

Model restored in epoch 1000.


In [6]:
# Load Raw Point-Clouds of class
n_pc_samples = 2048
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
pclouds_path = osp.join(pclouds_path, syn_id)
file_names = pio.load_filenames_of_input_data(pclouds_path, '.ply')
pclouds, model_ids, syn_ids = pio.load_crude_point_clouds(file_names=file_names, n_threads=50, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(pclouds), )

6778 files containing complete point clouds were found.


In [7]:
pcloud_data = PointCloudDataSet(pclouds, labels=model_ids, init_shuffle=False)
_, latent_codes, temp_names = latent_embedding_of_entire_dataset(pcloud_data, ae, ae_conf)
assert(np.alltrue(temp_names==model_ids))

In [8]:
top_image_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Meshes/Shape_Net_Core/2015_Summer_OUT/Images/'
top_image_dir = osp.join(top_image_dir, syn_id)
image_view_tag = 'image_p020_t337_r005.png'

In [9]:
n_thumbnails = 1000
rids = np.random.choice(range(pcloud_data.num_examples), n_thumbnails, replace=False)

In [10]:
# Find TSNE of sub-selected objects.
seed = 42
model = TSNE(n_components=2, random_state=seed, init='pca')
tsne_coords = model.fit_transform(latent_codes[rids])

In [11]:
# Make a list with the file-names of all to be used images.
image_files = []
for i in rids:
    image_files.append(osp.join(top_image_dir, model_ids[i], image_view_tag))

In [ ]:
plot_2d_embedding_in_grid_forceful_(tsne_coords, image_files, big_dim=13000, small_dim=400, save_file='tsne_1k_chairs_128bneck_.png');
# plot_2d_embedding_in_grid_greedy_way(tsne_coords, image_files, big_dim=14000, small_dim=400, save_file='tsne_greedy.png');
# plot_2d_embedding_in_grid_forceful(tsne_coords, image_files, big_dim=14000, small_dim=400, save_file='tsne_greedy.png');

In [13]:
import itertools
import numpy as np
import matplotlib.pylab as plt
import cv2
from PIL import Image

In [17]:
def _scale_2d_embedding(two_dim_emb):
    two_dim_emb -= np.min(two_dim_emb, axis=0)  # scale x-y in [0,1]
    two_dim_emb /= np.max(two_dim_emb, axis=0)
    return two_dim_emb

def read_transparent_png(filename):
    ''' TODO: add docstring
    SEE https://stackoverflow.com/questions/3803888/opencv-how-to-load-png-images-with-4-channels'''

    image_4channel = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    alpha_channel = image_4channel[:, :, 3]
    rgb_channels = image_4channel[:, :, :3]

    # White Background Image
    white_background_image = np.ones_like(rgb_channels, dtype=np.uint8) * 255

    # Alpha factor
    alpha_factor = alpha_channel[:, :, np.newaxis].astype(np.float32) / 255.0
    alpha_factor = np.concatenate((alpha_factor, alpha_factor, alpha_factor), axis=2)

    # Transparent Image Rendered on White Background
    base = rgb_channels.astype(np.float32) * alpha_factor
    white = white_background_image.astype(np.float32) * (1 - alpha_factor)
    final_image = base + white
    return final_image.astype(np.uint8)


def plot_2d_embedding_in_grid_forceful_(two_dim_emb, image_files, big_dim=2500, small_dim=200, save_file=None):
    x = _scale_2d_embedding(two_dim_emb)
    out_image = np.ones((big_dim, big_dim, 3), dtype='uint8') * 255
    N = two_dim_emb.shape[0]
    xnum = int(big_dim / float(small_dim))
    ynum = int(big_dim / float(small_dim))
    free = np.ones(N, dtype=np.bool)

    grid_2_img = np.ones((xnum, ynum), dtype='int') * -1
    res = float(small_dim) / float(big_dim)
    for i in xrange(xnum):
        for j in xrange(ynum):
            sorted_indices = np.argsort((x[:, 0] - i * res)**2 + (x[:, 1] - j * res)**2)
            possible = sorted_indices[free[sorted_indices]]

            if len(possible) > 0:
                picked = possible[0]
                free[picked] = False
                grid_2_img[i, j] = picked
            else:
                break

    for i in xrange(xnum):
        for j in xrange(ynum):
            if grid_2_img[i, j] > -1:
                im_file = image_files[grid_2_img[i, j]]
                
#                 fig = cv2.imread(im_file)
                fig = read_transparent_png(im_file)
                fig = cv2.resize(fig, (small_dim, small_dim))
                
                try:
                    out_image[i * small_dim:(i + 1) * small_dim, j * small_dim:(j + 1) * small_dim, :] = fig
                except:
                    print 'the code here fails. fix it.'
                    print im_file
                continue

    if save_file is not None:
        im = Image.fromarray(out_image)
        im.save(save_file)

    return out_image

In [ ]:
def random_samples_from_each_class(syn_ids, samples_per_class=100):
    selected = []
    for syn_id in np.unique(syn_ids):
        in_class = np.where(syn_ids == syn_id)[0]
        effective_samples = min(len(in_class), samples_per_class)
        selected += np.ndarray.tolist(np.random.choice(in_class, effective_samples, replace=False))
    return selected